In [1]:
import time
begin = time.ctime()
begin


'Mon Sep 27 12:49:47 2021'

In [2]:
import numpy as np
import pandas as pd
pd.set_option('max.rows',100)

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime

#sklearn
from sklearn.metrics import mean_squared_error

#Time Series
from pylab import rcParams
import statsmodels.api as sm

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss

from scipy.stats import boxcox

from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
#Params used

ip_path ='./Input/'
op_path ='./Output/'

rcParams['figure.figsize'] = 12, 8


#We have data from Sept 2015, Total months = 69
#Train data length
train_len = 64
#Data to be forecasted or test data length
fc_mths = 5

#Starting month
start_mth = 4 #4-starts from JaN 2016, 28 - starts from Jan 2018

exclude_mons = [] #['01-09-2015','01-10-2015']
exclude_mon_nums=[] #[20150901,20151001]

#Create an empty data frame with our date range with values filled as nan
start = pd.datetime(2015,9,1) #Date from which we have data
end = pd.datetime(2021,5,1) #Current Month  
index = pd.date_range(start, end,freq='MS')

# Create an empty data frame with our date range with values filled as nan
empty_ts = pd.DataFrame(index=index)
empty_ts['Issues'] = np.nan
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

from pmdarima.arima import auto_arima

In [3]:
#Params used

ip_path ='./Input/'
op_path ='./Output/'

rcParams['figure.figsize'] = 12, 8


#We have data from Sept 2015, Total months = 69
#Train data length
train_len = 64
#Data to be forecasted or test data length
fc_mths = 5

#Starting month
start_mth = 4 #4-starts from JaN 2016, 28 - starts from Jan 2018

exclude_mons = [] #['01-09-2015','01-10-2015']
exclude_mon_nums=[] #[20150901,20151001]

#Create an empty data frame with our date range with values filled as nan
start = pd.datetime(2015,9,1) #Date from which we have data
end = pd.datetime(2021,5,1) #Current Month  
index = pd.date_range(start, end,freq='MS')

# Create an empty data frame with our date range with values filled as nan
empty_ts = pd.DataFrame(index=index)
empty_ts['Issues'] = np.nan

In [4]:
def get_null_cnts(df_, thresh = None):
    null_cnts = df_.isnull().sum()
    null_cnts = null_cnts.sort_values(ascending=False)
    if thresh == None :
        return null_cnts
    else :
        return null_cnts[null_cnts > thresh]

In [5]:
def get_null_perc(df_, thresh = None):
    null_perc = round(100 * df_.isnull().sum()/len(df_), 2)
    null_perc = null_perc.sort_values(ascending=False)
    if thresh == None :
        return null_perc
    else :
        return null_perc[null_perc > thresh]

In [6]:
def get_group(x):
    if x in test_cids:
        return 'Test'
    else:
        return 'Control'

In [7]:
#Date handling functions

def convert_dt_str_to_mth(dt_str_):
    dt_ = datetime.strptime(dt_str_,'%d-%m-%Y %H:%M')
    return datetime.strftime(dt_,'%B%Y')

#Format 2020-09-01
def convert_dt_str_to_mth2(dt_str_):
    dt_ = datetime.strptime(dt_str_,'%Y-%m-%d')
    return datetime.strftime(dt_,'%B%Y')


#Date Format:20180701
def convert_dt_int_to_mth(dt_int_):
    dt_ = datetime.strptime(str(dt_int_),'%Y%m%d')
    return datetime.strftime(dt_,'%B%Y')

#Date Format:20180701
def convert_dt_int_to_dt(dt_int_):
    return datetime.strptime(str(dt_int_),'%Y%m%d')

def convert_mth_to_dt(mth_):
    return datetime.strptime(mth_,'%B%Y')

def add_months(dt_, num_):
    return dt_ + relativedelta(months=num_)

def convert_dt_to_mth(dt_):
    return datetime.strftime(dt_,'%B%Y')

def convert_mth_to_date_key(mth_):
    return datetime.strftime(convert_mth_to_dt(mth_),'%Y%m%d')

def get_mth_range(dt_):
    start = add_months(dt_,- (pre_post_months))
    end = add_months(dt_,(pre_post_months))
    dt_range = pd.date_range(start,end,freq='MS')
    mth_range = dt_range.map(convert_dt_to_mth)
    mth_range
    return mth_range

#This function is to get all previous months engaged prior to passed month. To derive if the customer is new in curent month
def get_prev_mths(mth_):
    return eng_dt_range[:eng_dt_range.index(mth_)]

In [8]:
me_df_orig = pd.read_excel('./Input/ME_Analysis.xlsx',sheet_name='Sheet1')

In [9]:
me_df_orig.shape

(29882, 12)

In [10]:
#me_df_orig.info()

print(me_df_orig['Editorial Language'].value_counts())

English                11792
(blank)                 8465
Spanish                 1917
German                  1897
French                  1521
Traditional Chinese     1038
Dutch                    749
Portuguese               747
Italian                  639
Swedish                  338
Danish                   298
Norwegian                220
Finnish                  182
Polish                    11
Hungarian                 11
Japanese                   9
Romanian                   9
Slovak                     6
Czech                      5
Catalan                    5
Croatian                   5
Greek                      4
Slovenian                  4
Estonian                   3
Bulgarian                  3
Lithuanian                 1
Turkish                    1
Latvian                    1
Albanian                   1
Name: Editorial Language, dtype: int64


In [11]:
me_df_orig.head(2)

,CalendarMonthStart,Editorial Language,Issues,EditorialCountryName,ComponentName,QueueCategory,QueueCategory1,QueueCategory2,Country&Lang,Country&Lang(T/F),Only Country,Only Lang
0,20160101,(blank),1,Australia,Appeals,NQP,AQ,NaN,Australia(blank),NaN,NaN,NaN
1,20160101,(blank),59,China,Appeals,NQP,AQ,NaN,China(blank),NaN,NaN,NaN


In [12]:
me_df = me_df_orig.copy()

In [13]:
me_df['Month'] = me_df.CalendarMonthStart.map(convert_dt_int_to_dt)

In [14]:
cats_df = me_df[['ComponentName','QueueCategory1','QueueCategory2']].drop_duplicates()

In [15]:
ts_core_comps = list(cats_df[(cats_df.QueueCategory1=='TS') & (cats_df.QueueCategory2=='Tscore')].ComponentName)


In [16]:
cats_df.shape

(42, 3)

In [17]:
me_df[ ((me_df.QueueCategory1=='TS') & (me_df.QueueCategory2.isnull()))].shape

(430, 13)

In [18]:
me_df = me_df[ ~((me_df.QueueCategory1=='TS') & (me_df.QueueCategory2.isnull()))]

In [19]:
me_df['Component'] = me_df.ComponentName.replace(ts_core_comps,'Tscore')

In [20]:
me_df.Component.value_counts()

Tscore                        8664
Phishing/Techscam             5808
Appeals                       4019
Complaints                    3480
Accounts & Billing            1841
Delivery                      1679
Supply TQ Investigation        948
Malware/Bundling/Downloads     820
Policy Questions               801
Enablements                    404
Matching                       314
Bing Places                    291
Bing Web Master Tools          205
SEAM                           178
Name: Component, dtype: int64

In [21]:
me_df = me_df.rename(columns={'EditorialCountryName':'Country'})

In [22]:
me_df = me_df[['Month','Country','Component','Issues']]

In [23]:
print('Number of countries:', len(me_df.Country.unique()), 'Number Of Components', len(me_df.Component.unique()))

Number of countries: 195 Number Of Components 14


In [24]:
cons_cons = ['Denmark', 'Finland', 'France', 'Norway', 'Spain', 'Sweden'] 
#cons_cons = ['Danish','Swedish','French','Norwegian','Sami','Spanish','Swedish']
#languges of these countries
#cons_cons = ['Denmark']
me_df

,Month,Country,Component,Issues
0,2016-01-01,Australia,Appeals,1
1,2016-01-01,China,Appeals,59
2,2016-01-01,China,Complaints,5
3,2016-01-01,China,Policy Questions,1
4,2016-01-01,France,Appeals,1
...,...,...,...,...
29877,2021-06-01,Croatia,Appeals,2
29878,2021-06-01,Croatia,Complaints,1
29879,2021-04-01,Bulgaria,Appeals,1
29880,2021-06-01,Bulgaria,Appeals,2


In [25]:
comp_me_df = me_df[me_df.Country.isin(cons_cons)]

In [26]:
print(comp_me_df)

           Month  Country         Component  Issues
4     2016-01-01   France           Appeals       1
18    2016-01-01  Denmark           Appeals      10
19    2016-01-01  Denmark        Complaints       5
20    2016-01-01  Denmark  Policy Questions       2
33    2016-01-01   France           Appeals       2
...          ...      ...               ...     ...
29682 2021-06-01    Spain            Tscore       1
29683 2021-06-01   Sweden            Tscore       1
29701 2021-06-01  Denmark            Tscore       1
29702 2021-06-01   France            Tscore       3
29703 2021-06-01   France            Tscore       2

[4472 rows x 4 columns]


In [27]:
comp_me_df.Country.value_counts()

France     1531
Spain      1061
Sweden      668
Denmark     526
Norway      403
Finland     283
Name: Country, dtype: int64

In [28]:
con_comp = comp_me_df[['Country','Component']].drop_duplicates().sort_values(['Country','Component'])
len(con_comp)
con_comp['FirstMonth'] = np.nan
con_comp['LastMonth'] = np.nan
con_comp['Total'] = 0
con_comp['Missing'] = 0
con_comp['LastCont'] = 0
con_comp['AvgIssues'] = 0

In [29]:
con_comp

,Country,Component,FirstMonth,LastMonth,Total,Missing,LastCont,AvgIssues
16847,Denmark,Accounts & Billing,NaN,NaN,0,0,0,0
18,Denmark,Appeals,NaN,NaN,0,0,0,0
29532,Denmark,Bing Places,NaN,NaN,0,0,0,0
19,Denmark,Complaints,NaN,NaN,0,0,0,0
17855,Denmark,Delivery,NaN,NaN,0,0,0,0
4544,Denmark,Enablements,NaN,NaN,0,0,0,0
10075,Denmark,Malware/Bundling/Downloads,NaN,NaN,0,0,0,0
5441,Denmark,Matching,NaN,NaN,0,0,0,0
8932,Denmark,Phishing/Techscam,NaN,NaN,0,0,0,0
20,Denmark,Policy Questions,NaN,NaN,0,0,0,0


In [30]:
for row in con_comp.itertuples():
    con, comp = row[1], row[2]
    temp_df = me_df[(me_df.Country == con) & (me_df.Component == comp)].groupby('Month')['Issues'].sum().reset_index()
    
    #First and Last month for which we have data for 
    first_mon = temp_df.Month.min()
    last_mon = temp_df.Month.max()
    
    #Month Range between first and last months 
    mth_range = pd.date_range( first_mon, last_mon, freq='MS')
    
    #Missing months in the range
    missing_mths = mth_range.difference(temp_df.Month)
    
    if len(missing_mths) > 0:
        #If data missing for a month or more, how many months data do we have continuosly from last month
        last_cont_mths = pd.date_range(max(missing_mths), last_mon, freq='MS')
    else:
        last_cont_mths = mth_range
    
    con_comp.loc[(con_comp.Country==con) & (con_comp.Component==comp),'FirstMonth'] = convert_dt_to_mth(first_mon)
    con_comp.loc[(con_comp.Country==con) & (con_comp.Component==comp),'LastMonth'] = convert_dt_to_mth(last_mon)
    con_comp.loc[(con_comp.Country==con) & (con_comp.Component==comp),'Total'] = len(mth_range)
    con_comp.loc[(con_comp.Country==con) & (con_comp.Component==comp),'Missing'] = len(missing_mths)
    con_comp.loc[(con_comp.Country==con) & (con_comp.Component==comp),'LastCont'] = len(last_cont_mths)-1 # Minus one
    
    #Average Issues
    con_comp.loc[(con_comp.Country==con) & (con_comp.Component==comp),'AvgIssues'] = temp_df.Issues.mean()

In [31]:
con_comp['MissingPerc'] = round((con_comp.Missing/con_comp.Total)*100)
con_comp['DataAvl'] = con_comp.Total - con_comp.Missing

In [32]:
con_comp.head(2)

,Country,Component,FirstMonth,LastMonth,Total,Missing,LastCont,AvgIssues,MissingPerc,DataAvl
16847,Denmark,Accounts & Billing,April2016,May2021,62,19,1,2.069767,31.0,43
18,Denmark,Appeals,January2016,June2021,66,0,65,15.318182,0.0,66


In [33]:
cons_con_comp = con_comp[con_comp.DataAvl > 15]
cons_con_comp.shape

(48, 10)

In [34]:
con_comp[~(con_comp.DataAvl > 15)]

,Country,Component,FirstMonth,LastMonth,Total,Missing,LastCont,AvgIssues,MissingPerc,DataAvl
29532,Denmark,Bing Places,June2021,June2021,1,0,0,1.000000,0.0,1
4544,Denmark,Enablements,April2019,April2021,25,13,2,1.833333,52.0,12
10075,Denmark,Malware/Bundling/Downloads,February2017,May2021,52,48,1,1.000000,92.0,4
5441,Denmark,Matching,October2019,January2021,16,11,1,1.800000,69.0,5
20,Denmark,Policy Questions,January2016,December2020,60,52,1,1.375000,87.0,8
20961,Finland,Bing Web Master Tools,June2018,June2018,1,0,0,1.000000,0.0,1
22712,Finland,Delivery,March2019,June2021,28,25,1,1.333333,89.0,3
9201,Finland,Malware/Bundling/Downloads,May2016,May2018,25,21,2,1.250000,84.0,4
5953,Finland,Matching,January2020,July2020,7,5,1,1.000000,71.0,2
63,Finland,Policy Questions,January2016,April2019,40,35,1,1.600000,88.0,5


In [35]:
recent_months = ['June2021','May2021','April2021']
display('Ignored Combinations are:', cons_con_comp[~cons_con_comp.LastMonth.isin(recent_months)])
cons_con_comp = cons_con_comp[cons_con_comp.LastMonth.isin(recent_months)]
cons_con_comp.shape

'Ignored Combinations are:'

,Country,Component,FirstMonth,LastMonth,Total,Missing,LastCont,AvgIssues,MissingPerc,DataAvl
9048,France,Malware/Bundling/Downloads,March2016,January2021,59,24,1,1.657143,41.0,35
70,France,Policy Questions,January2016,February2021,62,18,2,2.613636,29.0,44
9050,Spain,Malware/Bundling/Downloads,March2016,March2021,61,32,1,1.965517,52.0,29
17234,Sweden,Delivery,September2016,March2021,55,17,7,2.052632,31.0,38
11184,Sweden,Supply TQ Investigation,November2017,March2021,41,15,1,1.692308,37.0,26


(43, 10)

In [36]:
#Function to calculate error and prepare a data frame
def calc_error(method, act, pred):
    global cons_err_df
 
    #Root Mean Square Error
    rmse = np.sqrt(mean_squared_error(act, pred)).round(2)
    mae = np.mean(abs(act-pred))

#     cons_res_df['Actual_'+method] = results_df_['Issues'] #Not required for every method. Initiate it only once
    cons_res_df['Forecast_'+method] = pred
    
 
    #Mean Absolute Percentage error
    mape = np.round(np.mean(np.abs(act-pred)/act)*100,2)
    
    # Over and Under predictions to check the direction of the error
    over_pred, under_pred = [],[]
    
    
    #This is to avoid shown nan as average error
    if len(over_pred) == 0:
        over_pred_mape =0
    else:
#         over_pred_avg = round(np.mean(over_pred),1)
        preds = [item[0] for item in over_pred]
        acts = [item[1] for item in over_pred]
        over_pred_mape = np.round(np.mean(np.abs(np.array(preds)-np.array(acts))/np.array(acts))*100,2)

    if len(under_pred) == 0:
        under_pred_mape = 0
    else:
#         under_pred_avg = round(np.mean(under_pred),1)
        preds = [item[0] for item in under_pred]
        acts = [item[1] for item in under_pred]
        under_pred_mape = np.round(np.mean(np.abs(np.array(preds)-np.array(acts))/np.array(acts))*100,2)

    over_preds = str(len(over_pred)) + ' - ' + str(over_pred_mape)
    under_preds = str(len(under_pred)) + ' - ' + str(under_pred_mape)

 
    err_df_ = pd.DataFrame({'Method':[method], 'MAPE': [mape], 'RMSE': [rmse], 'MAE':[mae],
                           'Over Predictions':[over_preds], 'Under Predictions':[under_preds]})
    cons_err_df = cons_err_df.append(err_df_)
    
    #for i in [mape]:
        #print(i)
    
#     print('Method:', method, 'MAPE:', [mape], 'RMSE:', [rmse], 'MAE:',[mae])
    return err_df_

In [37]:
#Plot results of a time series forecasting model
def plot_results(method_name_, ts_data_train, results_df_):
    plt.figure(figsize=(12,4))
    plt.plot(ts_data_train, label='Train')
    plt.plot(results_df_['Issues'], label='Test')
    plt.plot(results_df_['Issues_FC'], label=method_name_+' Forecast')
    plt.legend(loc='best')
    plt.title(method_name_)
    plt.show()

In [38]:
    #Functions to run different models

    #Simple Exponential Smoothing
    def run_simple_exp_smooth(ts_data_train, ts_data_test, smoothing_level_=0.2, optimized_=False):
        model = SimpleExpSmoothing(ts_data_train)
        try:
            model_fit = model.fit(smoothing_level=smoothing_level_, optimized=optimized_)
            if debug:
                print(model_fit.params)
        except ValueError as e:
            print('Value Error while fitting the model SES:', str(e))
            return pd.DataFrame()

        results_df = ts_data_test.to_frame()
        results_df['Issues_FC'] = model_fit.forecast(len(ts_data_test))

        return results_df


    #Holts method with only trend
    def run_holt_trend(ts_data_train, ts_data_test, seasonal_periods_=12 , smoothing_level_=0.2, smoothing_slope_=0.01, optimized_=False):

        model = ExponentialSmoothing(np.asarray(ts_data_train) ,seasonal_periods = seasonal_periods_ , trend='additive', seasonal=None)

        try:
            model_fit = model.fit(smoothing_level = smoothing_level_, smoothing_slope = smoothing_slope_, optimized = optimized_)
            if debug:
                print(model_fit.params)
        except ValueError as e:
            print('Value Error while fitting the model Holt with only Trend:', str(e))
            return pd.DataFrame()

        results_df = ts_data_test.to_frame()
        results_df['Issues_FC'] = model_fit.forecast(len(ts_data_test))

        return results_df

    #Holt winters with trend and seasonality
    def run_holt_wint(ts_data_train, ts_data_test, seasonal_periods_=12 , trend_='add', seas_='add', optimized_=True, use_boxcox_=None,init_method_=None):

        model = ExponentialSmoothing(np.asarray(ts_data_train) ,seasonal_periods=seasonal_periods_ ,trend=trend_, seasonal=seas_, 
                                     initialization_method=init_method_, use_boxcox=use_boxcox_)

        try:
            model_fit = model.fit(optimized=optimized_)
            if debug:
                print(model_fit.params)
        except ValueError as e:
            print('Value Error while fitting the model Holt Winters:', str(e))
            return pd.DataFrame()

        results_df = ts_data_test.to_frame()
        results_df['Issues_FC'] = model_fit.forecast(len(ts_data_test))

        return results_df

    #ARIMA
    def run_arima(ts_data_train, ts_data_test, is_bc_=False, order_=(1,0,0)):

        model = ARIMA(ts_data_train, order=order_)
        try:
            model_fit = model.fit()
            if debug:
                print(model_fit.params)
        except ValueError as e:
            print('Value Error while fitting the model ARIMA:', str(e))
            return pd.DataFrame()

        results_df = ts_data_test.to_frame()
        results_df['Issues_FC'] = model_fit.forecast(len(ts_data_test))[0]

        if is_bc_:
            results_df['Issues'] = np.exp(results_df['Issues'])
            results_df['Issues_FC'] = np.exp(results_df['Issues_FC'])

        return results_df


    #SARIMAX
    def run_sarimax(ts_data_train, ts_data_test, is_bc_=False, order_=(1,1,1), seasonal_order_=(1,1,1,12), is_exog_=False, exog_=None):

        promo_train_, promo_test_ = None, None 
        if is_exog_:
            promo_train_ = exog_[0:len(ts_data_train)+1]
            promo_test_ = exog_[len(ts_data_train):]
            print(ts_data_train.shape, ts_data_test.shape, promo_train_.shape, promo_test_.shape)

        model = SARIMAX(ts_data_train, order=order_, seasonal_order=seasonal_order_,exog=promo_train_)

        try:
            model_fit = model.fit()
            if debug:
                print(model_fit.params)
        except ValueError as e:
            print('Value Error while fitting the model SARIMAX:', str(e))
            return pd.DataFrame()

        results_df = ts_data_test.to_frame()
        results_df['Issues_FC'] = model_fit.forecast(len(ts_data_test))

        if is_bc_:
            results_df['Issues'] = np.exp(results_df['Issues'])
            results_df['Issues_FC'] = np.exp(results_df['Issues_FC'])

        return results_df

    def run_auto_arima(ts_data_train, ts_data_test, is_bc_=False, start_p=1, start_q=1,max_d=2, max_p=3, max_q=3,start_P=0, 
                                 start_Q=0, max_P=3, max_D=1, max_Q=3, m=12, seasonal=True, trace=True, d=1, D=1, max_order = 5,
                                 maxiter = 50,
                                 error_action='warn', suppress_warnings=True, stepwise=True, random_state=20, n_fits=100, n_jobs = -1):

        # model = auto_arima(train, trace=True, error_action='ignore', suppress_warnings=True)
        model = auto_arima(ts_data_train,start_p=start_p,start_q=start_q,max_p=max_p, max_d=max_d, max_q=max_q,start_P=start_P, 
                                 start_Q=start_Q, max_P=max_P, max_D=max_D, max_Q=max_Q, m=m, seasonal=seasonal, trace=trace, d=d, 
                                 D=D, max_order = max_order, maxiter = maxiter, 
                                 error_action=error_action, suppress_warnings=suppress_warnings, 
                                 stepwise=stepwise, random_state=random_state, n_fits=n_fits, n_jobs = n_jobs)


        try:
            model.fit(ts_data_train)
            if debug:
                print(model_fit.params)
        except ValueError as e:
            print('Value Error while fitting the model Auto ARIMA:', str(e))
            return pd.DataFrame()

        results_df = ts_data_test.to_frame()

        try:
            results_df['Issues_FC'] = model.predict(len(ts_data_test))
        except ValueError as e:
            print('Value Error while prediction in Auto ARIMA:', str(e))
            return pd.DataFrame()

        if is_bc_:
            results_df['Issues'] = np.exp(results_df['Issues'])
            results_df['Issues_FC'] = np.exp(results_df['Issues_FC'])

        return results_df

In [39]:
def plot_calc_error(method, ts_data_train, results_df):
    if len(results_df) == 0:
        print('Results Dataframe is empty')
        return
    if debug:
        plot_results(method, ts_data_train, results_df)
    calc_error(method,results_df['Issues'],results_df['Issues_FC'])

#bc to indicate whether to boxcox transformation -- to indicate the model
def run_models(ts_data, bc, select='mape', run_all_models=None, run_Forecast_model=None):
    global results_df
    #Train test split - 80:20
    ts_len = len(ts_data)
    train_len = int(ts_len*.8)
    ts_data_train = ts_data[:train_len]
    ts_data_test = ts_data[train_len:]
    
    if bc:
        ts_data_bc = pd.Series(boxcox(ts_data.interpolate(), lmbda=0),name='Issues')
        #Index with all months from first to last
        idx = pd.date_range(ts_data.index.min(), ts_data.index.max(),freq='MS')
        #Reset index for boxcox data set
        ts_data_bc.index = idx

        ts_data_train_bc = ts_data_bc[:train_len]
        ts_data_test_bc = ts_data_bc[train_len:]
    
    cons_res_df['Actual'] = ts_data_test
    if run_all_models==True or run_Forecast_model =='SES':
        method = 'SES'
        results_df = run_simple_exp_smooth(ts_data_train, ts_data_test)
        print("---SES----")
        plot_calc_error(method, ts_data_train, results_df)
    
    if run_all_models==True or run_Forecast_model =='HoltsTrend':
        method = 'HoltsTrend'
        results_df = run_holt_trend(ts_data_train, ts_data_test)
        
        print("---HoltsTrend---")
        plot_calc_error(method, ts_data_train, results_df)
    
    if run_all_models==True or run_Forecast_model =='HWAdditive':
        method = 'HWAdditive'
        results_df = run_holt_wint(ts_data_train, ts_data_test)
        print("---HWAdditive-")
        
        plot_calc_error(method, ts_data_train, results_df)
    
    if run_all_models==True or run_Forecast_model in 'HWMult':
        if bc:
            method = 'HWMult'
            results_df = run_holt_wint(ts_data_train, ts_data_test,seas_='mul')
            print("---HWMult----")
            plot_calc_error(method, ts_data_train, results_df)
            
    if run_all_models==True or run_Forecast_model == 'ARIMA_(1, 1, 1)':

        ord = (1,1,1)
        method = 'ARIMA_'+str(ord)
        results_df = run_arima(ts_data_train, ts_data_test, order_=ord)
        
        print("---ARIMA_(1,1,1)----")
        
        plot_calc_error(method, ts_data_train, results_df)

    
    if bc:
        if run_all_models==True or run_Forecast_model == "ARIMA_BC_(1, 1, 1)":
            ord = (1,1,1)
            method = 'ARIMA_BC_'+str(ord)
            results_df = run_arima(ts_data_train_bc, ts_data_test_bc, is_bc_=True, order_=ord) #TODO - pass Box Cox data
            print("---ARIMA_BC_(1,1,1)----")
            plot_calc_error(method, ts_data_train, results_df)

        if run_all_models==True or run_Forecast_model == "SARIMA_BC_(1, 1, 1)_(1, 1, 1, 12)":
            ord_1 = (1,1,1)
            ord_2 = (1,1,1,12)
            method = 'SARIMA_BC_'+str(ord_1)+'_'+str(ord_2)
            results_df = run_sarimax(ts_data_train_bc, ts_data_test_bc, is_bc_=True, order_=ord_1, seasonal_order_=ord_2)#TODO
            print("---SARIMA_BC_(1,1,1)_(1,1,1,12)----")
            plot_calc_error(method, ts_data_train, results_df)

        if run_all_models==True or run_Forecast_model == "SARIMA_BC_(1, 2, 1)_(1, 1, 1, 12)":
            ord_1 = (1,2,1)
            ord_2 = (1,1,1,12)
            method = 'SARIMA_BC_'+str(ord_1)+'_'+str(ord_2)
            results_df = run_sarimax(ts_data_train_bc, ts_data_test_bc, is_bc_=True, order_=ord_1, seasonal_order_=ord_2) #TODO
            print("---SARIMA_BC_(1,2,1)_(1,1,1,12)----")
            plot_calc_error(method, ts_data_train, results_df)

        if run_all_models==True or run_Forecast_model == "SARIMA_BC_(1, 2, 1)_(0, 1, 1, 12)":
            ord_1 = (1,2,1)
            ord_2 = (0,1,1,12)
            method = 'SARIMA_BC_'+str(ord_1)+'_'+str(ord_2)
            results_df = run_sarimax(ts_data_train_bc, ts_data_test_bc, is_bc_=True, order_=ord_1, seasonal_order_=ord_2) #TODO
            print("---SARIMA_BC_(1,2,1)_(0,1,1,12)----")
            plot_calc_error(method, ts_data_train, results_df)

        if run_all_models==True or run_Forecast_model == "SARIMA_(1, 2, 1)_(1, 1, 1, 12)":
            ord_1 = (1,2,1)
            ord_2 = (1,1,1,12)
            method = 'SARIMA_'+str(ord_1)+'_'+str(ord_2)
            results_df = run_sarimax(ts_data_train, ts_data_test, order_=ord_1, seasonal_order_=ord_2)
        
            print("---SARIMA_(1,2,1)_(1,1,1,12)----")
            plot_calc_error(method, ts_data_train, results_df)

        if run_all_models==True or run_Forecast_model == "SARIMA_(1, 2, 1)_(0, 1, 1, 12)":
            ord_1 = (1,2,1)
            ord_2 = (0,1,1,12)
            method = 'SARIMA_'+str(ord_1)+'_'+str(ord_2)
            results_df = run_sarimax(ts_data_train, ts_data_test, order_=ord_1, seasonal_order_=ord_2)
        
            print("---SARIMA_(1,2,1)_(0,1,1,12)----")
            plot_calc_error(method, ts_data_train, results_df)

    if run_all_models==True or run_Forecast_model =='PMD_Diff1':
        
        method = 'PMD_Diff1'
        results_df = run_auto_arima(ts_data_train, ts_data_test, is_bc_=False, start_p=1, start_q=1,max_p=8, max_q=8,start_P=0, 
                                 start_Q=0, max_P=8, max_Q=8, m=12, seasonal=True, trace=debug, d=1, D=1, 
                                 error_action='warn', suppress_warnings=True, stepwise=True, random_state=20, n_fits=100)
        
        print("---PMD_Diff1----")
        plot_calc_error(method, ts_data_train, results_df)

    if run_all_models==True or run_Forecast_model =='PMD_Diff2':
        method = 'PMD_Diff2'
        results_df = run_auto_arima(ts_data_train, ts_data_test, is_bc_=False, start_p=1, start_q=1,max_p=8, max_d=3, max_q=8,start_P=0, 
                                 start_Q=0, max_P=8, max_Q=8, m=12, seasonal=True, trace=debug, d=2, 
                                 error_action='warn', suppress_warnings=True, stepwise=True, random_state=20, n_fits=100)
        
        print("---PMD_Diff2----")
        plot_calc_error(method, ts_data_train, results_df)

    if run_all_models==True or run_Forecast_model =='PMD_Diff1_BC':
        if bc:
            method = 'PMD_Diff1_BC'
            results_df = run_auto_arima(ts_data_train_bc, ts_data_test_bc, is_bc_=True, start_p=1, start_q=1,max_p=8, max_q=8,start_P=0, 
                                     start_Q=0, max_P=8, max_Q=8, m=12, seasonal=True, trace=debug, d=1, D=1, 
                                     error_action='warn', suppress_warnings=True, stepwise=True, random_state=20, n_fits=100)
            
            print("---PMD_Diff1_BC----")
            plot_calc_error(method, ts_data_train, results_df)
            
    if not run_all_models :        
        display(results_df)


In [40]:
%%time
#Forecasting with missing values as 0
debug = False

all_res_dfs = {}
all_err_dfs = {}
i = 0 
minErrorModel = []
for row in cons_con_comp[['Country','Component']].itertuples():
    con, comp = row[1], row[2]
    print('Building models for Country:{} and Component:{}'.format(con, comp))
    print('='*100)
    ts_data = me_df[(me_df.Country==con)&(me_df.Component==comp)].groupby('Month')['Issues'].sum()
    
    #Index with all months from first to last
    idx = pd.date_range(ts_data.index.min(), ts_data.index.max(),freq='MS')
    #Reindex time series data to fill values with 0
    ts_data = ts_data.reindex(idx, fill_value=0)
    
    print('TS Data Set shape: ',ts_data.shape)

    #Initiate Consolidated Result and Error dataframes
    cons_err_df = pd.DataFrame()
    cons_res_df = pd.DataFrame()

    run_models(ts_data, bc=False, run_all_models=True)
    
    
    all_res_dfs[(con, comp)] = cons_res_df
    all_err_dfs[(con, comp)] = cons_err_df
    
    display(cons_err_df)
    
    #display("----minimum error ------",cons_err_df['RMSE'].min())
    for i in cons_err_df.itertuples():
        #print(i[3])
        if i[3] == cons_err_df['RMSE'].min() :
            #print("mehod---",i[1])
            minErrorModel.append(i[1])
            
            break
            
    #display(cons_res_df)
    if debug:
        display(cons_res_df)
        

    cons_con_comp['Min Error Method'] = minErrorModel




Building models for Country:Denmark and Component:Accounts & Billing
TS Data Set shape:  (62,)
---SES----
---HoltsTrend---
---HWAdditive-
---ARIMA_(1,1,1)----
---PMD_Diff1----
---PMD_Diff2----


,Method,MAPE,RMSE,MAE,Over Predictions,Under Predictions
0,SES,inf,1.00,0.874954,0 - 0,0 - 0
0,HoltsTrend,inf,7.34,6.967840,0 - 0,0 - 0
0,HWAdditive,inf,1.41,1.100947,0 - 0,0 - 0
0,"ARIMA_(1, 1, 1)",inf,1.10,0.910991,0 - 0,0 - 0
0,PMD_Diff1,inf,2.05,1.648214,0 - 0,0 - 0
0,PMD_Diff2,inf,8.93,8.017480,0 - 0,0 - 0


ValueError: Length of values (1) does not match length of index (43)

In [41]:
minErrorModel

['SES']

In [42]:
cons_con_comp

,Country,Component,FirstMonth,LastMonth,Total,Missing,LastCont,AvgIssues,MissingPerc,DataAvl
16847,Denmark,Accounts & Billing,April2016,May2021,62,19,1,2.069767,31.0,43
18,Denmark,Appeals,January2016,June2021,66,0,65,15.318182,0.0,66
19,Denmark,Complaints,January2016,June2021,66,1,54,21.661538,2.0,65
17855,Denmark,Delivery,March2017,June2021,52,23,1,1.689655,44.0,29
8932,Denmark,Phishing/Techscam,January2016,June2021,66,5,22,3.918033,8.0,61
11595,Denmark,Supply TQ Investigation,February2018,June2021,41,20,2,1.809524,49.0,21
16671,Denmark,Tscore,January2016,June2021,66,19,8,2.957447,29.0,47
17756,Finland,Accounts & Billing,March2017,June2021,52,34,1,1.333333,65.0,18
61,Finland,Appeals,January2016,June2021,66,1,59,6.723077,2.0,65
62,Finland,Complaints,January2016,June2021,66,3,47,17.650794,5.0,63


In [43]:
#for forecasting the values

#Forecasting with missing values as 0
debug = False
bc = False

all_res_dfs = {}
all_err_dfs = {}
i = 0 
minErrorModel = []

#display(cons_con_comp)
for row in cons_con_comp[['Country','Component','Min Error Method']].itertuples():
    #print(row)
    con, comp = row[1], row[2]
    print('Building models for Country:{} and Component:{}'.format(con, comp))
    print('='*100)
    ts_data = me_df[(me_df.Country==con)&(me_df.Component==comp)].groupby('Month')['Issues'].sum()
    
    #Index with all months from first to last
    idx = pd.date_range(ts_data.index.min(), ts_data.index.max(),freq='MS')
    #Reindex time series data to fill values with 0
    ts_data = ts_data.reindex(idx, fill_value=0)
    
    print('TS Data Set shape: ',ts_data.shape)

    #Initiate Consolidated Result and Error dataframes
    cons_err_df = pd.DataFrame()
    cons_res_df = pd.DataFrame()

    run_models(ts_data, bc=False, run_all_models=False,run_Forecast_model=row[3])
    
    all_res_dfs[(con, comp)] = cons_res_df
    all_err_dfs[(con, comp)] = cons_err_df
    
    
   
    #display(cons_res_df)
    if debug:
        display(cons_res_df)
        
        
   

KeyError: "['Min Error Method'] not in index"

In [ ]:
%%time
#Forecasting with missng values linear interpolation
debug = False

all_res_intp_dfs = {}
all_err_intp_dfs = {}
i = 0
for row in cons_con_comp[['Country','Component']].itertuples():
    con, comp = row[1], row[2]
    print('Building models for Country:{} and Component:{}'.format(con, comp))
    print('='*100)
    ts_data = me_df[(me_df.Country==con)&(me_df.Component==comp)].groupby('Month')['Issues'].sum()
    
    #Index with all months from first to last
    idx = pd.date_range(ts_data.index.min(), ts_data.index.max(),freq='MS')
    #Reindex time series data to fill values with nan and use interpolation
    ts_data = ts_data.reindex(idx, fill_value=np.nan)
    #Interploate to impute
    ts_data = ts_data.interpolate()
    
    print('TS Data Set shape: ',ts_data.shape)

    #Initiate Consolidated Result and Error dataframes
    cons_err_df = pd.DataFrame()
    cons_res_df = pd.DataFrame()

    run_models(ts_data, bc=True,run_all_models=True)
    
    
    all_res_intp_dfs[(con, comp)] = cons_res_df
    all_err_intp_dfs[(con, comp)] = cons_err_df
    
    display(cons_err_df)
    
    for i in cons_err_df.itertuples():
        #print(i[3])
        if i[3] == cons_err_df['RMSE'].min() :
            #print("mehod---",i[1])
            minErrorModel.append(i[1])
            
            break
    
    if debug:
        display(cons_res_df)
        
    #For testing.Run only limited combos
#     i += 1
#     if i==1:
#         breark

cons_con_comp['Min Error Method2'] = minErrorModel

In [ ]:
cons_con_comp

In [ ]:
%%time
#Forecasting with missng values linear interpolation
debug = False

all_res_intp_dfs = {}
all_err_intp_dfs = {}
i = 0
for row in cons_con_comp[['Country','Component','Min Error Method2']].itertuples():
    con, comp = row[1], row[2]
    print('Building models for Country:{} and Component:{}'.format(con, comp))
    print('='*100)
    ts_data = me_df[(me_df.Country==con)&(me_df.Component==comp)].groupby('Month')['Issues'].sum()
    
    #Index with all months from first to last
    idx = pd.date_range(ts_data.index.min(), ts_data.index.max(),freq='MS')
    #Reindex time series data to fill values with nan and use interpolation
    ts_data = ts_data.reindex(idx, fill_value=np.nan)
    #Interploate to impute
    ts_data = ts_data.interpolate()
    
    print('TS Data Set shape: ',ts_data.shape)

    #Initiate Consolidated Result and Error dataframes
    cons_err_df = pd.DataFrame()
    cons_res_df = pd.DataFrame()

    run_models(ts_data, bc=True,run_all_models=False,run_Forecast_model=row[3])
    
    
    all_res_intp_dfs[(con, comp)] = cons_res_df
    all_err_intp_dfs[(con, comp)] = cons_err_df
    
    display(cons_err_df)
    if debug:
        display(cons_res_df)

    #For testing.Run only limited combos
#     i += 1
#     if i==1:
#         break
    
    print()